In [1]:
import sys
!{sys.executable} -m pip install scipy==1.2.1 ssm tqdm==4.46.0

     |████████████████████████████████| 24.8MB 17.5MB/s 
     |████████████████████████████████| 317kB 12.8MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
  Created wheel for ssm: filename=ssm-0.0.1-cp37-cp37m-linux_x86_64.whl size=858035 sha256=899a0465c93f45d2da569c093c51f737654830bc3667cb5a364c531413f4bde6
  Stored in directory: /home/idies/.cache/pip/wheels/7a/26/7f/279606a509bdc1b2bd8659632f406e1c5a8fe58b733a28c791
  Created wheel for autograd: filename=autograd-1.3-cp37-none-any.whl size=47990 sha256=bd16457503e3430621badf3320135407a087b103265f494c77cbc7d0c3830add
  Stored in directory: /home/idies/.cache/pip/wheels/42/62/66/1121afe23ff96af4e452e0d15e68761e3f605952ee075ca99f
Successfully built ssm autograd
ERROR: spaghetti 1.4.1 requires watermark, which is not installed.
ERROR: libpysal 4.2.2 requires bs4, which is not installed.
ERROR: quilt3 3.1.10 has requirement ruamel.yaml>=0.15.78, but you'll have ruamel-yaml 0.15.46 which is incompatible.
ERROR: giddy 2.3.0 ha

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ssm
import seaborn as sns
import scipy
from scipy.stats import entropy

In [2]:
df=pd.read_csv("sample.csv")

In [3]:
df.head()

,Unnamed: 0,day,weekday,wakeup,firstMeal,lastMeal,sleep
0,0,1,Monday,5.94,8.05,17.83,21.08
1,1,2,Tuesday,6.02,7.99,17.87,21.16
2,2,3,Wednesday,6.19,8.22,18.05,21.14
3,3,4,Thursday,5.99,8.08,18.11,20.95
4,4,5,Friday,5.83,8.06,17.78,20.75


In [4]:
df.columns

Index(['Unnamed: 0', 'day', 'weekday', 'wakeup', 'firstMeal', 'lastMeal',
       'sleep'],
      dtype='object')

### add noise

In [5]:
# add noise
List = ['wakeup', 'firstMeal', 'lastMeal','sleep']

def addnoise(df, N):   
    Df=df[[ 'wakeup', 'firstMeal', 'lastMeal','sleep']]   
    nn=len(Df['wakeup'])
    for i in List:
        Df[i] = Df[i] + np.random.normal(0, N, nn)
    return Df  

### set time bins

In [6]:
def setTB(bins, dd):
    
    Newdd=pd.DataFrame(columns=("time",'wakeup', 'firstMeal', 'lastMeal','sleep'))
    
    for i in List:
        
        counts, division = np.histogram(np.array(dd[i]), range=(0, 24), bins=bins)
        
        Newdd[i]=counts

    Newdd["time"]=division[0 : -1]
    
    return Newdd

### Fit model with different latent states

In [7]:
## obs:dataframe
def testsample(train, test, num_ls):
    trainx=np.array(train[List])
    testx=np.array(test[List])
    
    num_states = num_ls
    num_neurons = 4
    
    hmm = ssm.HMM(num_states, num_neurons, observations="poisson")
    
    N_iters = 1000
    lls = hmm.fit(trainx, method='em', num_em_iters=N_iters) 
    
    zpre=hmm.most_likely_states(testx) #discrete states of test sets
    
    samm=[]
    
    for i in range(len(zpre)):
        samm.append(hmm.observations.sample_x(zpre[i], testx[i], with_noise=True))
    testsamples=np.array([list(i) for i in samm])
    
    return testsamples

## KL

In [30]:
def KL(testdata, tsample):
    # change counts to time points
    tp_obs=[]
    tp_sam=[]
    for i in range(len(List)):
        tp_obs.append(np.repeat(np.array(pop2["time"]), np.array(testdata[:,i])))
        tp_sam.append(np.repeat(np.array(pop2["time"]), np.array(tsample[:,i])))   
    
    #combining data from all labels
    total_obs=[]
    total_sam=[]
    
    for i in range(len(List)):
        for j in range(len(tp_obs[i])):
            total_obs.append(tp_obs[i][j])
            
    for i in range(len(List)):
        for j in range(len(tp_sam[i])):
            total_sam.append(tp_sam[i][j])       
    
    #calculate kl
    ttime=Jtime
    act = total_obs
    sam = total_sam
    actual_density = scipy.stats.kde.gaussian_kde(act)
    actual_density = actual_density(ttime)
    sample_density = scipy.stats.kde.gaussian_kde(sam)
    sample_density = sample_density(ttime)
    kl=(entropy(actual_density, qk=sample_density)+entropy(sample_density, qk=actual_density))/2
    
    return kl #, total_obs, total_sam, tp_obs, tp_sam, actual_density, sample_density

# MAE & KL

In [31]:
Jdf = pd.read_csv('gpar_derived.csv')
Jtime = pd.read_csv('gpar_derived.csv')["time"]
Jtime = np.array(Jtime)

In [32]:
def MAE(Sam, Odata):

#     mae=0
#     for i in range(len(List)):
#         s=Sam[:, i]
#         o=Odata[:, i]
#         mae = mae + sum(abs(s-o))/50

    mae=sum(abs(Sam.sum(axis = 1)-Odata.sum(axis = 1)))/50    
    
    return mae

In [33]:
Mae = []
Kl = []
pop = addnoise(df, 0.1)     
pop2 = setTB(50, pop) 
xx = np.array(pop2[List], dtype = int) 

/Users/jingyiren/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [34]:
num_states = 4
num_neurons = 12
#fit model
hmm = ssm.HMM(num_states, num_neurons, observations="poisson")

N_iters = 1000
lls = hmm.fit(xx, method='em', num_em_iters=N_iters) 

zpre=hmm.most_likely_states(xx) #discrete states of test sets

for j in range(100):
    
    samm=[]
    for i in range(len(zpre)):
        samm.append(hmm.observations.sample_x(zpre[i], xx[i], with_noise=True))  # generate sample
        
    sam_pop=np.array([list(i) for i in samm])
    Mae.append(MAE(xx, sam_pop))        
    Kl.append(KL(xx, sam_pop)) 

In [35]:
Mean_Mae = np.mean(Mae)
Mean_Kl = np.mean(Kl)
Mae=sorted(Mae)
Kl=sorted(Kl)
maeCI_a=Mae[5]  
maeCI_b=Mae[95]  
klCI_a=Kl[5] 
klCI_b=Kl[95] 

In [36]:
print("HMM : ")
print('mean of KL : ' + str(Mean_Kl))
print("CI for KL : " + "["+str(klCI_a)+", "+str(klCI_b)+"]")
print('mean of MAE : ' + str(Mean_Mae))
print("CI for MAE : " + "["+str(maeCI_a)+", "+str(maeCI_b)+"]")

HMM : 
mean of KL : 0.15601875427815443
CI for KL : [0.13111728970528672, 0.18586569113541312]
mean of MAE : 11.912999999999997
CI for MAE : [10.96, 12.78]
